### Baisc Model Testing

Code to test the baseline functionality of various models provided by the transformers library. 

This includes llama-3-Korean-Bllossom-8B, the EEVE model by yanolja, the Synatra model, and others.

This notebook is to test the basic capabilities and functionalities of the model, and doesn't not involve any specialized methods such as training, fine-tuning, RAG, etc.

In [1]:
# Import modules 

from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
import torch

/home/kimjaehyun/anaconda3/envs/nlp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set model name for loading from the transformers library

# Yanolja/EEVE
model_name = "yanolja/EEVE-Korean-Instruct-10.8B-v1.0"
tokenizer_name = "yanolja/EEVE-Korean-Instruct-10.8B-v1.0"

# # maywell/Synatra-V0.1-7B, maywell/Synatra-7B-Instruct-v0.2
# model_name = "maywell/Synatra-7B-Instruct-v0.2"
# tokenizer_name = "maywell/Synatra-7B-Instruct-v0.2"

# # openchat/openchat_3.5
# model_name = "openchat/openchat_3.5"
# tokenizer_name = "openchat/openchat_3.5"

# # monologg/kobert
# model_name = "monologg/kobert"
# tokenizer_name = "monologg/kobert"

In [3]:
# Load model and tokenizer

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16, # half-precision to speed up inference
)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name,
                                          device_map="auto")

Loading checkpoint shards: 100%|██████████| 5/5 [00:10<00:00,  2.03s/it]


In [4]:
# Set text and prompt template

# # Yanolja/EEVE prompt
prompt_template = "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.\nHuman: {prompt}\nAssistant:\n"

# # maywell/Synatra prompt
# prompt_template = "<s>[INST] {prompt}. [/INST]"

# # OpenChat prompt
# prompt_template = "GPT4 Correct User: Hello<|end_ofturn|>GPT4 Correct Assistant: Hi<|end_of_turn|>GPT4 Correct User: {prompt}<|end_of_turn|>GPT4 Correct Assistant:\n"

# text = "한국의 수도는 어디인가요? 아래 선택지 중 골라주세요.\n\n(A) 경성\n(B) 부산\n(C) 평양\n(D) 서울\n(E) 전주"
text = """당신은 판사입니다. 다음과 같은 청구취지와 항소취지를 보고, 편결요지를 내십시오.

1. 청구취지

가. 본소

원고(반소피고, 이하 원고라 한다)와 피고(반소원고, 이하 피고라 한다)는 이혼한다.

나. 반소

원고의 본소 청구가 인용되는 것을 조건으로, 원고는 피고에게, 위자료로 1억 원 및 이에 대하여 반소장 송달 다음 날부터 제1심판결 선고일까지는 연 5%의, 그 다음 날부터 다 갚는 날까지는 연 20%의 각 비율에 의한 금원을, 재산분할로 514,800,000원 및 이에 대하여 이 판결 확정일 다음 날부터 다 갚는 날까지 연 5%의 비율에 의한 금원을 각 지급하라.

2. 항소취지

제1심판결 중 본소 부분을 취소하고, 원고의 본소 청구를 기각한다. 원고의 본소 청구가 인용되는 것을 조건으로, 제1심판결의 반소 부분 중 아래에서 추가로 지급을 명하는 부분에 해당하는 피고 패소 부분을 취소한다. 원고는 피고에게, 위자료로 20,000,000원 및 이에 대하여 이 사건 예비적 반소장 송달 다음 날부터 당심판결 선고일까지는 연 5%의, 그 다음 날부터 다 갚는 날까지는 연 20%의 각 비율에 의한 금원을, 재산분할로 499,800,000원 및 이에 대하여 이 판결 확정일 다음 날부터 다 갚는 날까지 연 5%의 비율에 의한 금원을, 각 지급하라.

3. 부대항소취지

제1심판결의 반소 중 원고에 대하여 위자료로 50,000,000원을 초과하여 지급을 명한 원고 패소 부분과 재산분할에 관한 원고 패소 부분을 취소하고, 그 취소 부분에 해당하는 피고의 반소 청구를 기각한다.
"""

In [5]:
# Process through model and obtain model output
with torch.no_grad():
    model_inputs = tokenizer(prompt_template.format(prompt=text), return_tensors="pt").to("cuda")

    model.generation_config.pad_token_id = model.generation_config.eos_token_id  # To prevent early ending of sentence
    outputs = model.generate(**model_inputs, max_new_tokens=2048, no_repeat_ngram_size=5)
    # outputs = model.generate(**model_inputs, pad_token_id=tokenizer.eos_token_id)
output_text = tokenizer.batch_decode(outputs, skip_special_token=True)[0]
print(output_text)

<s> A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.
Human: 당신은 판사입니다. 다음과 같은 청구취지와 항소취지를 보고, 편결요지를 내십시오.

1. 청구취지

가. 본소

원고(반소피고, 이하 원고라 한다)와 피고(반소원고, 이하 피고라 한다)는 이혼한다.

나. 반소

원고의 본소 청구가 인용되는 것을 조건으로, 원고는 피고에게, 위자료로 1억 원 및 이에 대하여 반소장 송달 다음 날부터 제1심판결 선고일까지는 연 5%의, 그 다음 날부터 다 갚는 날까지는 연 20%의 각 비율에 의한 금원을, 재산분할로 514,800,000원 및 이에 대하여 이 판결 확정일 다음 날부터 다 갚는 날까지 연 5%의 비율에 의한 금원을 각 지급하라.

2. 항소취지

제1심판결 중 본소 부분을 취소하고, 원고의 본소 청구를 기각한다. 원고의 본소 청구가 인용되는 것을 조건으로, 제1심판결의 반소 부분 중 아래에서 추가로 지급을 명하는 부분에 해당하는 피고 패소 부분을 취소한다. 원고는 피고에게, 위자료로 20,000,000원 및 이에 대하여 이 사건 예비적 반소장 송달 다음 날부터 당심판결 선고일까지는 연 5%의, 그 다음 날부터 다 갚는 날까지는 연 20%의 각 비율에 의한 금원을, 재산분할로 499,800,000원 및 이에 대하여 이 판결 확정일 다음 날부터 다 갚는 날까지 연 5%의 비율에 의한 금원을, 각 지급하라.

3. 부대항소취지

제1심판결의 반소 중 원고에 대하여 위자료로 50,000,000원을 초과하여 지급을 명한 원고 패소 부분과 재산분할에 관한 원고 패소 부분을 취소하고, 그 취소 부분에 해당하는 피고의 반소 청구를 기각한다.

Assistant:
판사로서, 청구취지와 항소에 근거하여 다음과 같은 편결요지를 내리겠습니다:


In [ ]:
# (Nearly) Identical operation as except tokens are generated and printed one by one
with torch.no_grad():
    model_inputs = tokenizer(prompt_template.format(prompt=text), return_tensors="pt")#.to("cuda")
    output_ids = model_inputs['input_ids']
    while True:
        outputs = model.generate(output_ids, max_new_tokens=1, early_stopping=True)
        new_token_id = outputs[0, -1:]
        output_ids = torch.cat([output_ids, new_token_id.unsqueeze(0)], dim=-1)
        
        # Decode and print the latest token
        new_token = tokenizer.decode(new_token_id, skip_special_tokens=True)
        print(new_token, end=' ', flush=True)
        
        # Stop if the generated token is eos_token
        if new_token_id.item() == tokenizer.eos_token_id:
            break

Next lets try to test the Blossom 8B model, as provided in the HuggingFace library:

https://huggingface.co/MLP-KTLim/llama-3-Korean-Bllossom-8B

In [3]:
# Using transformers pipeline for Blossom 8B

import transformers
import torch

model_id = "MLP-KTLim/llama-3-Korean-Bllossom-8B"

model = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

model.model.eval()

PROMPT = '''You are a helpful AI assistant. Please answer the user's questions kindly. 당신은 유능한 AI 어시스턴트 입니다. 사용자의 질문에 대해 친절하게 답변해주세요.'''

messages = [
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{text}"}
    ]

prompt = model.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
)

terminators = [
    model.tokenizer.eos_token_id,
    model.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model(
    prompt,
    max_new_tokens=2048,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9
)

print(outputs[0]["generated_text"][len(prompt):])

Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.13s/it]


### 판결문

**재판장**  
[판사 이름]

**[일자]**  
[판결번호]  
[소송번호]  
[소송이름]  
[소송구성]

**판결문**

1. 본소 청구는 기각한다.
2. 반소 청구는 기각한다.

**이유**
1. 본소 청구에 대한 판결:
제1심판결의 본소 부분은 원고가 피고에게 위자료로 1억 원 및 이에 대하여 반소장 송달 다음 날부터 제1심판결 선고일까지는 연 5%의, 그 다음 날부터 다 갚는 날까지는 연 20%의 각 비율에 의한 금원을, 재산분할로 514,800,000원 및 이에 대하여 이 판결 확정일 다음 날부터 다 갚는 날까지 연 5%의 비율에 의한 금원을 각 지급하라는 조건으로 인용되는 것을 조건으로, 원고는 피고에게 위와 같은 금원을 지급하라는 명령을 내렸다. 제1심판결의 본소 부분은 위의 조건을 충족하는 경우에만 원고의 청구를 인용하는 것이므로, 원고가 위 조건을 충족하지 못한 상태에서 본소 청구를 인용할 수 없기 때문에 본소 청구는 기각한다.

2. 반소 청구에 대한 판결:
제1심판결의 반소 부분은 원고에게 위자료로 1억 원 및 이에 대하여 반소장 송달 다음 날부터 제1심판결 선고일까지는 연 5%의, 그 다음 날부터 다 갚는 날까지는 연 20%의 각 비율에 의한 금원을, 재산분할로 514,800,000원 및 이에 대하여 이 판결 확정일 다음 날부터 다 갚는 날까지 연 5%의 비율에 의한 금원을 각 지급하라는 명령을 내렸다. 원고가 위 조건을 충족하지 못한 상태에서 반소 청구를 인용할 수 없기 때문에 반소 청구는 기각한다.

**결론**
이와 같은 이유로, 본소 청구는 기각하고, 반소 청구는 기각한다.

**판사 서명**
[판사 이름]  
[판사 서명]  
[일자]
